In [1]:
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import time
import datetime
from ast import literal_eval

Daily meteorological information obtained from https://open-meteo.com/

In [2]:
def generate_link(df):
    lat = df.centroid_lat
    lon = df.centroid_lon
    date_initial = '2013-01-01'
    date_last = str(datetime.datetime.now()-datetime.timedelta(days=1)).split(' ')[0]
    meteorological_features = [
        'weathercode',
        'temperature_2m_max',
        'temperature_2m_min',
        'temperature_2m_mean',
        'apparent_temperature_max',
        'apparent_temperature_min',
        'apparent_temperature_mean',
        'sunrise',
        'sunset',
        'shortwave_radiation_sum',
        'precipitation_sum',
        'rain_sum',
        'precipitation_hours',
    ]
    meteorological_features_link_text = ','.join(meteorological_features)
    
    link_parts = [
        'https://archive-api.open-meteo.com/v1/archive?',
        f'latitude={lat}&longitude={lon}',
        f'&start_date={date_initial}&end_date={date_last}&',
        f'daily={meteorological_features_link_text}&timezone=GMT',
    ]
    complete_link = ''.join(link_parts)
    return complete_link

In [3]:
data = pd.read_pickle('../data/raw/muni.pkl')

In [4]:
past_meteodata = pd.read_csv('../data/raw/meteorological_information.csv', index_col=0)

In [6]:
past_meteodata['CVE_CONCATENADA'] = past_meteodata['CVEGEO'].astype('str').str.zfill(5)

In [28]:
wanted_links = ['CVE_CONCATENADA', 'data_link', 'centroid_lat', 'centroid_lon', 'meteorological_information']
past_meteodata = past_meteodata[wanted_links]

In [29]:
past_meteodata['meteorological_information'] = past_meteodata['meteorological_information'].str.replace("'", '"').str.replace('None', '-1')

/tmp/ipykernel_131/1477791224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  past_meteodata['meteorological_information'] = past_meteodata['meteorological_information'].str.replace("'", '"').str.replace('None', '-1')


In [31]:
meteorological_information = past_meteodata['meteorological_information'].apply(json.loads)

In [33]:
past_meteodata['meteorological_information'] = meteorological_information

/tmp/ipykernel_131/3706486870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  past_meteodata['meteorological_information'] = meteorological_information


In [34]:
past_meteodata.to_pickle('../data/raw/historical_meteorological_information.pkl')

In [ ]:
d = pd.read_pickle('../data/raw/historical_meteorological_information.pkl')

In [ ]:
d

In [13]:
centroids = data.centroid#.to_crs(epsg='4326')
centroids_lon = centroids.apply(lambda x:x.coords[0][0])
centroids_lat = centroids.apply(lambda x:x.coords[0][1])
data['centroid_lat'] = centroids_lat
data['centroid_lon'] = centroids_lon

In [7]:
#!pip install openmeteo-requests
#!pip install requests-cache retry-requests

import openmeteo_requests
import pandas as pd
import requests_cache

from retry_requests import retry
from openmeteo_sdk.Variable import Variable

In [2]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [3]:
params = {
    "latitude": 21.811436,
    "longitude": -102.295803,
    "start_date":"2013-01-01",
    "end_date":"2024-01-01",
    "daily": [
        'temperature_2m_max',
        'temperature_2m_min',
        'temperature_2m_mean',
        'apparent_temperature_max',
        'apparent_temperature_min',
        'apparent_temperature_mean',
        'sunrise',
        'sunset',
        'shortwave_radiation_sum',
        'precipitation_sum',
        'rain_sum',
        'precipitation_hours',
    ],
}
url = "https://archive-api.open-meteo.com/v1/archive"


In [5]:
responses = openmeteo.weather_api(url, params=params)
response = responses[0]
daily = response.Daily()
daily_variables = list(
    map(lambda i: daily.Variables(i), range(0, daily.VariablesLength()))
)

In [8]:
meteorological_info = pd.DataFrame()
dates = pd.date_range(
    start=pd.to_datetime(daily.Time(), unit='s'),
    end=pd.to_datetime(daily.TimeEnd(), unit='s'),
    freq=pd.Timedelta(seconds=daily.Interval()),
    inclusive='left'
)
meteorological_info['date'] = dates

for i, element in enumerate(params['daily']):
    meteorological_info[element] = daily_variables[i].ValuesAsNumpy()

In [36]:
def get_daily_response(df, date_initial='2013-01-01'):
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    
    lat = df.centroid_lat
    lon = df.centroid_lon
    date_last = str(datetime.datetime.now()-datetime.timedelta(days=5)).split(' ')[0]
    meteorological_features = [
        'temperature_2m_max',
        'temperature_2m_min',
        'temperature_2m_mean',
        'apparent_temperature_max',
        'apparent_temperature_min',
        'apparent_temperature_mean',
        'sunrise',
        'sunset',
        'shortwave_radiation_sum',
        'precipitation_sum',
        'rain_sum',
        'precipitation_hours',
    ]
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date":date_initial,
        "end_date":date_last,
        "daily": meteorological_features,
    }
    url = "https://archive-api.open-meteo.com/v1/archive"
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    daily = response.Daily()
    #time.sleep((5*60)/2463)
    return daily

In [35]:
data.apply(get_daily_response, axis=1)

OpenMeteoRequestsError: {'error': True, 'reason': 'Minutely API request limit exceeded. Please try again in one minute.'}

#The WGS84 Coordinate Systems adds Greenwich as the starting point (prime meridian) for the longitude (0°) and sets the units in degrees (°). This coordinate system also has a unique reference code, the so-called EPSG code, which is 4326


In [5]:
data_link = data.apply(generate_link, axis=1)

In [6]:
data_link.iloc[0]

'https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT'

In [8]:
def obtain_link_data(link):
    link_response = requests.get(link)
    my_json = link_response.content.decode('utf8')
    link_data = json.loads(my_json)
    if link_data.get('latitude'):
        return link_data
    else:
        print(f'Try again... for link: {link}')
        obtain_link_data(link)
        time.sleep(10)

In [9]:
#test1 = obtain_link_data(data_link.iloc[0])

In [10]:
informational_return = data_link.apply(obtain_link_data)

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=20

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=20

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=20

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.900694487634112&longitude=-102.70490393952089&start_date=2013-01-01&end_date=20

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=22.146634607006842&longitude=-102.52583871922944&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=22.146634607006842&longitude=-102.52583871922944&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=22.146634607006842&longitude=-102.52583871922944&start_date=2013-01-01&end_date=20

...

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&dai

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&dai

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&dai

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=19.4740717379277&longitude=-89.6096439688366&start_date=2013-01-01&end_date=2023-12-31&dai

RecursionError: maximum recursion depth exceeded while calling a Python object

In [11]:
informational_return

NameError: name 'informational_return' is not defined

In [15]:
informational_return = data_link.apply(obtain_link_data)

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=20

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=20

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=20

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=20

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=20

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=20

Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=2023-12-15&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours&timezone=GMT
Try again... for link: https://archive-api.open-meteo.com/v1/archive?latitude=21.811436232601537&longitude=-102.29580290235445&start_date=2013-01-01&end_date=20


KeyboardInterrupt



In [48]:
data['data_link'] = data_link

In [49]:
data['meteorological_information'] = informational_return

In [56]:
data[data.meteorological_information.apply(lambda x: x.get('latitude')).fillna(0)>0]

,CVE_ENT,CVE_MUN,CVEGEO,NOM_ENT,AREA,PERIMETER,NOM_MUN,COV_,COV_ID,geometry,centroid_lat,centroid_lon,data_link,meteorological_information
0,01,001,01001,Aguascalientes,116635.92455,1166.35925,Aguascalientes,0,1,"POLYGON ((-102.09775 22.02325, -102.09857 22.0...",21.811436,-102.295803,https://archive-api.open-meteo.com/v1/archive?...,"{'latitude': 21.827766, 'longitude': -102.2895..."
85,07,006,07006,Chiapas,25562.20129,255.62201,Amatenango de la Frontera,85,86,"POLYGON ((-91.99007 15.63576, -91.99033 15.635...",15.556835,-92.097163,https://archive-api.open-meteo.com/v1/archive?...,"{'latitude': 15.5711775, 'longitude': -92.1166..."
180,07,103,07103,Chiapas,17601.61410,176.01614,TuzantÃ¡n,180,181,"POLYGON ((-92.33392 15.20180, -92.33423 15.201...",15.140441,-92.411564,https://archive-api.open-meteo.com/v1/archive?...,"{'latitude': 15.1493845, 'longitude': -92.5257..."
273,09,005,09005,Ciudad de MÃ©xico,8737.83072,87.37831,Gustavo A. Madero,273,274,"POLYGON ((-99.11789 19.59059, -99.11860 19.584...",19.504065,-99.115864,https://archive-api.open-meteo.com/v1/archive?...,"{'latitude': 19.507908, 'longitude': -99.11618..."


In [94]:
desired_meteorological_information_cols = ['CVEGEO', 'CVE_ENT', 'CVE_MUN', 'NOMGEO', 'centroid_lat', 'centroid_lon', 'data_link', 'meteorological_information']
data[desired_meteorological_information_cols].to_csv('../data/raw/meteorological_information.csv')